## Setup

In [2]:
# Allows us to edit submodules on the fly without needing to restart kernel
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.environ["OMP_NUM_THREADS"] = "4"  # Safe and stable, avoid MKL memory leak on Windows when backtesting

In [3]:
# Imports
from common_imports import *
from api_init import initialize_apis
from data_extraction import get_continuous_risk_free_rate, get_live_ric_data
from testing import split_train_test_calendar_window, compare_simulated_to_real_returns, run_rolling_volatility_regime_model
from plotting import *
from branding import *
from models.gbm import monte_carlo_paths_gbm
from models.jump import monte_carlo_paths_jumps, calibrate_jump_model
from models.regime import monte_carlo_paths_regimes, calibrate_volatility_regime_model
from models.heston import monte_carlo_paths_heston, calibrate_heston_model

In [20]:
# Constants
PRICE = 5000 # In real time, this is equal to (bid+ask)/2 = mid-price
SIGMA = 0.2 # Annualised volatility
R = 0.04 # CONTINUOUS risk-free rate proxy for 1 year
Q = 0.01 # CONTINUOUS dividend yield of the asset
T = 1.0 # 1 year
STEPS = 252 # Assume simulation is for 1 trading year
N_PATHS = 100_000 # For all but the regime models, this number should be >= 100,000. The regime model should be lower
RIC = 'SPY' # The default asset to analyse
OPTION_RIC = 'SPYATMIV.U' # Options of an asset are a different asset class in Refinitiv
WINDOW = 63 # The number of days in 3 trading months
EXPECTED_RETURN = 0.0651637109089667 # Expected Return of the S&P500 according to Actinver on 02/01/2024
SEED = 42 # Ensures consistency in results
FONT = 'Century Gothic' # Default Actinver font
KNOCK_IN = 0.8
AC_TRIGGER = 1.0
sns.set_palette(COLOR_CYCLE) # Sets colours of graphs to be Actinver colors 

In [ ]:
# API set-up 
fred, eikon = initialize_apis()

## Extraction

In [ ]:
# Gets the current values of input for our models - only run if you want live data
r, _, _ = get_continuous_risk_free_rate(fred)
S0, sigma, q = get_live_ric_data(RIC, OPTION_RIC)

In [5]:
# Read README for more details on this database
historic_data = pd.read_excel("Data/historic_data_spx.xlsx")
historic_data["Date"] = pd.to_datetime(historic_data["Date"])
historic_data = historic_data.set_index("Date")
historic_data_prices = historic_data['Price']

In [6]:
# Split into train and test
train_data, test_data = split_train_test_calendar_window(historic_data, 
                                      test_start_date='01/01/2024', 
                                      train_years=5, 
                                      date_column='Date')    

In [11]:
# Main
train_price_series = train_data['Price']
test_price_series = test_data['Price']

# Define initial parameters
S0 = train_data['Price'].iloc[-1]            # Last observed price before simulation, last price of the training date
q  = train_data['Div'].iloc[-1]              # Last dividend yield
r  = train_data['Risk_Free'].iloc[-1]        # Last risk-free rate
expected_return = train_data['Actinver_Expected_Return'].iloc[-1] # Last expected return
last_train_date = train_data.index[-1] # The simulation should start ON the first day of the train set, which means it's initial starting point is the last_train_date

# Prepend last_train_date to test_dates for the graphing 
paths_dates = pd.concat([pd.Series([last_train_date]), pd.Series(test_data.index)], ignore_index=True)

## Models

In [21]:
# GBM, standard vanilla one
gbm_paths = monte_carlo_paths_gbm(
    S0=S0, q=q, r=r, expected_return=expected_return,
    sigma=SIGMA, T=T, steps=STEPS, n_paths=N_PATHS,
    measure='Real', seed=SEED
)

Simulating GBM Paths: 100%|█████████████████████████████████████████████████████████| 252/252 [00:01<00:00, 180.99it/s]


In [ ]:
# Jumps, worst performing one thus far
jump_sigma, lambda_j, m_j, s_j, jumps = calibrate_jump_model(train_price_series)

jump_paths = monte_carlo_paths_jumps(
    S0=S0,   # start at the most recent price
    q=q,                     # dividend yield
    r=r,                     # risk-free rate
    expected_return=expected_return,       # real-world drift if measure='Real'
    T=1.0,                        # 1 year
    steps=STEPS,                  # daily steps
    n_paths=N_PATHS,              # number of paths
    sigma=jump_sigma,             # diffusion volatility from calibration
    lambda_j=lambda_j,          # jump intensity
    m_j=m_j,                    # mean log jump size
    s_j=s_j,                    # std dev log jump size
    measure='Real',             # or 'Real'
    seed=SEED                     # for reproducibility
)

In [ ]:
# Volatility regime switching, best performing one thus far, but also, the slowest, at 1,000,000 paths it takes 44 seconds per iteration and needs to do 252 for a year, so it would take 3 hours
sigmas, transition_matrix_df, regimes_series = calibrate_volatility_regime_model(
    price_series=train_price_series,
    n_regimes=4,               
    annualize=True,
    verbose=True,
    export_excel=True
)

regime_paths, regimes = monte_carlo_paths_regimes(S0, q, r, expected_return, T, STEPS, 10_000,
                                          sigmas, transition_matrix_df,
                                          measure='Real',
                                          init_probs=None,
                                          seed=SEED)

np.savez_compressed(
    "regime_mc_simulation.npz",
    regime_paths=regime_paths,
    regimes=regimes
)

Regarding the number of regimes:

2 regimes provided poor performance

3 regimes performed exceptionally well

4 regimes may be overfitting

5 regimes delivered good but less robust results than 3

These results were obtained using a 5-year training window, tested on data from the year 2024. Again, one might argue the validity, but speed is king. We can try more regimes if time allows

In [ ]:
v0, kappa, theta, sigma_v, rho = calibrate_heston_model(train_price_series)

heston_paths, v_paths = monte_carlo_paths_heston(
    S0=S0,     # Start from most recent price
    q=q,                       # Dividend yield
    r=r,                       # Risk-free rate
    expected_return=expected_return,         # If using 'Real' measure
    T=T,                        # 1 year horizon
    steps=STEPS,                    # Daily steps
    n_paths=N_PATHS,                # Number of paths
    v0=0.02,                        # From calibration
    kappa=4,
    theta=0.02,
    sigma_v=0.35,
    rho=-0.7,
    measure='Real',       # or 'Real'
    seed=SEED
)

## Additional Tests

In [ ]:
max_iterations = 10_000  # safety cap
threshold = 0.05  # significance level

count = 0
ks_pval = 0  # initialize with a non-significant p-val

while ks_pval < threshold and count < max_iterations:
    ks_stat, ks_pval, ad_stat, ad_sig_level, log_returns_sim_flat = compare_simulated_to_real_returns(
        real_prices=test_price_series,
        simulated_paths=jump_paths,
        asset_name="SPX",
        verbose=None,
        seed=None
    )

    count += 1
    print(ks_pval)

print(f"Stopped after {count} iterations")
print(f"Final KS p-value: {ks_pval:.6f}")

Note: We simulate over 100,000 paths, which produces millions of individual return observations. Running a KS test directly between this large simulated sample and the comparatively small real return series (251 points) would lead to an unbalanced comparison and increase the risk of spurious significance purely due to sample size differences.

To address this, the code randomly samples 251 log returns from the full set of simulated returns, matching the number of real observations. This process is repeated 100 times with different random seeds to capture variability. In 91 out of 100 trials, the Kolmogorov–Smirnov (KS) test produced a p-value above 0.05, indicating that we could not reject the null hypothesis that real and simulated returns come from the same distribution.

In parallel, we also applied the Anderson–Darling (AD) k-sample test, which is more sensitive to discrepancies in the tails of the distribution. In some cases where the KS test did not reject the null, the AD test did, suggesting potential mismatches in extreme-value behavior (fat tails, skewness, or kurtosis) that are not captured by the KS test alone.

Given time constraints, we adopted this bootstrapped KS/AD testing procedure as a statistically sound approach for validating distributional similarity, with the AD test serving as a complementary diagnostic for tail accuracy.

In essence, an improvement in the model would require better tail-risk modelling

## Graphs and Results

In [ ]:
plot_return_histogram_comparison(
    real_prices=test_price_series,
    simulated_paths=gbm_paths,
    asset_name="SPX",
    use_log_returns=False,
    seed=SEED,
    bins=100,
    show_title=False,
    show_axis_titles=False,
    show_legend=False,
    filename="gbm_histogram",
    fixed_xlim=(-0.1, 0.125)
)

In [ ]:
plot_return_kde_comparison(
    real_prices=test_price_series,
    simulated_paths=gbm_paths,
    asset_name="SPX",
    use_log_returns=False,
    seed=SEED,
    show_title=False,
    show_axis_titles=False,
    show_legend=False,
    filename="gbm_kde",
    fixed_xlim=(-0.04, 0.04)
)

In [ ]:
plot_return_qq(
    real_prices=test_price_series,
    simulated_paths=gbm_paths,
    asset_name="SPX",
    use_log_returns=False,
    seed=SEED,
    show_title=False,
    show_axis_titles=False,
    show_legend=False,
    filename="gbm_qq"
)

In [ ]:
plot_return_ecdf_comparison(
    real_prices=test_price_series,
    simulated_paths=gbm_paths,
    asset_name="SPX",
    use_log_returns=False,
    seed=SEED,
    show_title=False,
    show_axis_titles=False,
    show_legend=False,
    filename="gbm_ecdf"
)

In [ ]:
global_ymin, global_ymax = compute_global_ylim(gbm_paths, jump_paths, regime_paths, heston_paths,
                                              low=1, high=99)
fixed_ylim = (global_ymin, global_ymax)

In [ ]:
plot_fan_chart(
    paths=gbm_paths,
    percentiles=[5, 25, 50, 75, 95],
    colors=sns.color_palette("Blues", n_colors=3),
    filename_prefix="gbm_fan_chart",
    line_color=GOLD_1,
    dates=paths_dates,
    show_axis_titles=False,
    fixed_ylim=fixed_ylim
)

## Backtesting
So the objective is to get some sort of Mark To Market by using the calculator we had.
We need the lkogic so that we calibrate the volatility regime model every DAY
Once we have the paths, we feed it into the calculator, and we backsolve for value, assuming we have a coupon:

In [9]:
# Create a new series just for the backtesting, i dont know
price_series_for_backtesting = pd.Series(
    data=historic_data["Price"].values,
    index=pd.to_datetime(historic_data["Date"]),
    name="Price"
)

In [22]:
calibration_path = 'data/rolling_4regime_calibration_S&P500.xlsx'
if os.path.exists(calibration_path):
    print("✅ Loading cached calibration results...")
    calibration_results = pd.read_excel(calibration_path, index_col="Date", parse_dates=True)
else:
    print("🚀 Running calibration...")
    calibration_results = run_rolling_volatility_regime_model(price_series_for_backtesting, 
                                                              export_excel=True, 
                                                              save_path=calibration_path)

✅ Loading cached calibration results...


In [37]:
# Get last row of calibration
latest = calibration_results.iloc[-1]

# Extract required inputs
sigmas = [latest[f"Sigma_{i}"] for i in range(4)]  # for 4 regimes
transition_matrix = latest.filter(like="Transition_").values.reshape(4, 4)

# Market inputs
S0 = historic_data_prices.iloc[-1]
r = historic_data["Risk_Free"].iloc[-1]
q = historic_data["Div"].iloc[-1]
expected_return = 0.08

In [1]:
def monte_carlo_paths_regimes_with_stats(
    S0, q, r, expected_return, T, steps, n_paths,
    sigmas, transition_matrix,
    measure='Risk-Neutral',
    init_probs=None,
    seed=None,
    autocall_barrier=None,
    autocall_freq=None,
    knockin_barrier=None,
):
    """
    Simulate GBM paths with N-state volatility regime switching and compute autocall/knock-in stats.

    Returns:
    - paths: ndarray of shape (steps + 1, n_paths)
    - regimes: ndarray of shape (steps, n_paths)
    - stats: dict with autocall and knock-in metrics
    """
    if seed is not None:
        np.random.seed(seed)

    sigmas = np.asarray(sigmas)
    n_regimes = len(sigmas)
    transition_matrix = np.asarray(transition_matrix)

    if init_probs is None:
        init_probs = np.full(n_regimes, 1 / n_regimes)
    else:
        init_probs = np.asarray(init_probs)

    dt = T / steps
    mu = r - q if measure == 'Risk-Neutral' else expected_return

    paths = np.empty((steps + 1, n_paths))
    regimes = np.empty((steps, n_paths), dtype=int)

    paths[0] = S0
    current_regime = np.random.choice(n_regimes, size=n_paths, p=init_probs)
    alive = np.ones(n_paths, dtype=bool)
    autocall_flags = np.zeros(n_paths, dtype=bool)
    autocall_counts = []

    for t in range(1, steps + 1):

        # This is where the beauty happens 
        transition_probs = transition_matrix[current_regime]
        rand_draws = np.random.uniform(size=n_paths) 
        cumulative_probs = np.cumsum(transition_probs, axis=1) # Convert each path's transition probabilities into cumulative probabilities (CDF) This is required for inverse transform sampling
        next_regime = (rand_draws[:, None] < cumulative_probs).argmax(axis=1) # Determine the next regime for each path. The regime is the first one where the cumulative probability exceeds the random draw
        current_regime = next_regime
        regimes[t - 1] = current_regime

        # Standard model construction
        sigma_t = sigmas[current_regime]
        z = np.random.standard_normal(n_paths)
        drift = (mu - 0.5 * sigma_t**2) * dt
        diffusion = sigma_t * np.sqrt(dt) * z
        growth = np.exp(drift + diffusion)
        paths[t] = paths[t - 1] * growth

        if autocall_barrier is not None and autocall_freq and t % autocall_freq == 0:
            triggered = (paths[t] >= S0 * autocall_barrier) & (~autocall_flags)
            autocall_counts.append(np.sum(triggered))
            autocall_flags |= triggered
            alive[triggered] = False

        if not alive.any():
            paths[t + 1:] = np.nan
            break

    not_called = ~autocall_flags
    final_prices = paths[-1]
    barrier_abs = knockin_barrier * S0 if knockin_barrier is not None else -np.inf

    ki_mask = not_called & (final_prices <= barrier_abs)
    expected_ST_given_KI = np.mean(final_prices[ki_mask]) if np.any(ki_mask) else None
    mat_below_ki = np.sum(ki_mask)
    mat_above_ki = np.sum(not_called & (final_prices > barrier_abs))

    stats = {f"AC{i+1}": count / n_paths for i, count in enumerate(autocall_counts)}
    stats.update({
        "Maturity Above KI": mat_above_ki / n_paths,
        "Maturity Below KI": mat_below_ki / n_paths,
        "Expected ST Given KI": expected_ST_given_KI
    })

    return paths, regimes, stats

In [ ]:
all_results = []

for date, row in tqdm(calibration_results.iterrows(), total=len(calibration_results), desc="Simulating for each date"):
    print(f"Number of results: {len(all_results)}")

    try:
        # Extract parameters for this date
        n_regimes = 4  # or infer from row
        sigmas = [row[f"Sigma_{i}"] for i in range(n_regimes)]
        transition_matrix = row.filter(like="Transition_").values.reshape(n_regimes, n_regimes)

        # Inputs for Monte Carlo
        S0 = historic_data_prices.loc[date]  # starting price that day
        r = historic_data["Risk_Free"].loc[date]
        q = historic_data["Div"].loc[date]
        expected_return = historic_data["Actinner_Expected_Return"].loc[date]  # typo fixed


        # Get last row of calibration
        latest = calibration_results.iloc[-1]
        
        # Extract required inputs
        sigmas = [latest[f"Sigma_{i}"] for i in range(4)]  # for 4 regimes
        transition_matrix = latest.filter(like="Transition_").values.reshape(4, 4)
        
        # Market inputs
        S0 = historic_data_prices.iloc[-1]
        r = historic_data["Risk_Free"].iloc[-1]
        q = historic_data["Div"].iloc[-1]
        expected_return = 0.08

        # Run regime-switching simulation
        paths, regimes, stats = monte_carlo_paths_regimes_with_stats(
            S0=S0,
            q=q,
            r=r,
            expected_return=expected_return,
            T=1.0,
            steps=252,
            n_paths=1000,  # adjust as needed
            sigmas=sigmas,
            transition_matrix=transition_matrix,
            measure="Risk-Neutral",
            seed=SEED,  # or vary with time if needed
            autocall_barrier=1.0,
            autocall_freq=None,
            knockin_barrier=0.8,
        )

        # Store summary (mean final price, std, quantiles)
        final_prices = paths[-1, :]
        all_results.append({
            "Date": date,
            "Simulated_Mean": final_prices.mean(),
            "Simulated_Std": final_prices.std(),
            "Simulated_P5": np.percentile(final_prices, 5),
            "Simulated_P95": np.percentile(final_prices, 95),
            "Simulated_Median": np.median(final_prices),
        })

        print(f"Number of results: {len(all_results)}")


    except Exception as e:
        print(f"⚠️ Skipping {date} due to error: {e}")
        continue

# Convert to DataFrame
sim_summary_df = pd.DataFrame(all_results).set_index("Date")

In [47]:
type(calibration_results)

pandas.core.frame.DataFrame

In [ ]:
all_results = []


# date 

for date, row in tqdm(calibration_results.iterrows(), total=len(calibration_results), desc="Simulating for each date"):
    try:
        n_regimes = 4
        sigmas = [row[f"Sigma_{i}"] for i in range(n_regimes)]
        transition_matrix = row.filter(like="Transition_").values.reshape(n_regimes, n_regimes)
        S0 = price_series_for_backtesting.loc[date]
        r = 0.05
        q = 0.01
        expected_return=0.08

        print(date)

        # Run regime-switching simulation
        paths, regimes = monte_carlo_paths_regimes(
            S0=S0,
            q=q,
            r=r,
            expected_return=expected_return,
            T=1.0,
            steps=252,
            n_paths=100,
            sigmas=sigmas,
            transition_matrix=transition_matrix,
            measure="Risk-Neutral",
            seed=SEED
        )

        final_prices = paths[-1, :]
        all_results.append({
            "Date": date,
            "Simulated_Mean": final_prices.mean(),
            "Simulated_Std": final_prices.std(),
            "Simulated_P5": np.percentile(final_prices, 5),
            "Simulated_P95": np.percentile(final_prices, 95),
            "Simulated_Median": np.median(final_prices),
        })
    except Exception as e:
        print(f"⚠️ Skipping {date} due to error: {e}")
        continue

# Convert to DataFrame
sim_summary_df = pd.DataFrame(all_results).set_index("Date")

In [ ]:
# After this, historic_data will be indexed

In [34]:
historic_data["Date"] = pd.to_datetime(historic_data["Date"])
historic_data_indexed = historic_data.set_index("Date")

In [35]:
historic_data_prices

0       1283.72
1       1285.20
2       1280.09
3       1282.46
4       1270.84
         ...   
4852    5844.19
4853    5886.55
4854    5892.58
4855    5916.93
4856    5958.38
Name: Price, Length: 4857, dtype: float64

In [33]:
all_results = []

for date, row in tqdm(calibration_results.iterrows(), total=len(calibration_results), desc="Simulating for each date"):
    try:
        n_regimes = 4
        sigmas = [row[f"Sigma_{i}"] for i in range(n_regimes)]
        transition_matrix = row.filter(like="Transition_").values.reshape(n_regimes, n_regimes)

        # Check if required market inputs exist for this date
        print(date)
        print(historic_data.index)
        if date not in historic_data_prices.index or date not in historic_data.index:
            print(f"⚠️ Skipping {date} due to missing market data")
            continue

        # Market inputs
        S0 = historic_data_prices.loc[date]
        r = historic_data["Risk_Free"].loc[date]
        q = historic_data["Div"].loc[date]
        expected_return = historic_data["Actinver_Expected_Return"].loc[date]  # or fix typo

        # Run simulation
        paths, regimes, stats = monte_carlo_paths_regimes_with_stats(
            S0=S0,
            q=q,
            r=r,
            expected_return=expected_return,
            T=1.0,
            steps=252,
            n_paths=1000,
            sigmas=sigmas,
            transition_matrix=transition_matrix,
            measure="Risk-Neutral",
            seed=SEED,
            autocall_barrier=1.0,
            autocall_freq=None,
            knockin_barrier=0.8,
        )

        print(S0, r, q, expected_return)

        # Store simulation summary
        final_prices = paths[-1, :]
        all_results.append({
            "Date": date,
            "Simulated_Mean": final_prices.mean(),
            "Simulated_Std": final_prices.std(),
            "Simulated_P5": np.percentile(final_prices, 5),
            "Simulated_P95": np.percentile(final_prices, 95),
            "Simulated_Median": np.median(final_prices),
        })

    except Exception as e:
        print(f"⚠️ Skipping {date} due to error: {e}")
        continue

# Build DataFrame
if all_results:
    sim_summary_df = pd.DataFrame(all_results).set_index("Date")
    print(sim_summary_df.head())
else:
    print("❌ No simulation results generated.")


Simulating for each date:  25%|██▍       | 886/3597 [00:00<00:00, 4291.93it/s]

2011-01-28 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2011-01-28 00:00:00 due to missing market data
2011-01-31 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2011-01-31 00:00:00 due to missing market data
2011-02-01 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2011-02-01 00:00:00 due to missing market data
2011-02-02 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2011-02-02 00:00:00 due to missing market data
2011-02-03 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2011-02-03 00:00:00 due to missing market data
2011-02-04 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2011-02-04 00:00:00 due to missing market data
2011-02-07 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2011-02-07 00:00:00 due to missing market data
2011-02-08 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2011-02-08 00:00:00 due to missing market data
2011-02-09 00:00:00
RangeIndex(start=0, stop=4857, step=

Simulating for each date:  62%|██████▏   | 2240/3597 [00:00<00:00, 3597.81it/s]

2016-08-30 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2016-08-30 00:00:00 due to missing market data
2016-08-31 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2016-08-31 00:00:00 due to missing market data
2016-09-01 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2016-09-01 00:00:00 due to missing market data
2016-09-02 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2016-09-02 00:00:00 due to missing market data
2016-09-06 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2016-09-06 00:00:00 due to missing market data
2016-09-07 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2016-09-07 00:00:00 due to missing market data
2016-09-08 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2016-09-08 00:00:00 due to missing market data
2016-09-09 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2016-09-09 00:00:00 due to missing market data
2016-09-12 00:00:00
RangeIndex(start=0, stop=4857, step=

Simulating for each date: 100%|█████████▉| 3590/3597 [00:00<00:00, 4185.58it/s]

2019-12-23 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2019-12-23 00:00:00 due to missing market data
2019-12-24 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2019-12-24 00:00:00 due to missing market data
2019-12-26 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2019-12-26 00:00:00 due to missing market data
2019-12-27 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2019-12-27 00:00:00 due to missing market data
2019-12-30 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2019-12-30 00:00:00 due to missing market data
2019-12-31 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2019-12-31 00:00:00 due to missing market data
2020-01-02 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2020-01-02 00:00:00 due to missing market data
2020-01-03 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2020-01-03 00:00:00 due to missing market data
2020-01-06 00:00:00
RangeIndex(start=0, stop=4857, step=

Simulating for each date: 100%|██████████| 3597/3597 [00:00<00:00, 4042.61it/s]

2025-05-08 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2025-05-08 00:00:00 due to missing market data
2025-05-09 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2025-05-09 00:00:00 due to missing market data
2025-05-12 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2025-05-12 00:00:00 due to missing market data
2025-05-13 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2025-05-13 00:00:00 due to missing market data
2025-05-14 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2025-05-14 00:00:00 due to missing market data
2025-05-15 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2025-05-15 00:00:00 due to missing market data
2025-05-16 00:00:00
RangeIndex(start=0, stop=4857, step=1)
⚠️ Skipping 2025-05-16 00:00:00 due to missing market data
❌ No simulation results generated.


In [27]:
historic_data.head()

,Date,Price,IV,Risk_Free,Div,Actinver_Expected_Return
0,2006-01-27,1283.72,0.136327,0.044105,0.018607,0.116700
1,2006-01-30,1285.20,0.136373,0.044257,0.018424,0.117329
2,2006-01-31,1280.09,0.138087,0.044162,0.019291,0.122860
3,2006-02-01,1282.46,0.137092,0.044544,0.018838,0.121704
4,2006-02-02,1270.84,0.140062,0.044563,0.018784,0.132723


## Miscellaneous

In [ ]:
blue_palette = sns.color_palette("Blues", n_colors=10)

In [ ]:
blue_palette

In [ ]:
# Blended for the number of paths
actinver_palette_blended = blend_palette(ACTINVER_BLUES, n_colors=10)

In [ ]:
actinver_palette_blended